<a href="https://colab.research.google.com/github/clam004/case/blob/main/Load_Training_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount to my google drive which is where ive stored the data
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/case
!ls

Mounted at /content/drive
/content/drive/MyDrive/case
ACES.ipynb		  modelstates  T0_3B_dialog_summarization.ipynb
data			  __pycache__  Training_Data.xlsx
Load_Training_Data.ipynb  sol.ipynb    utils.py


In [7]:
import pandas as pd

df = pd.read_excel('Training_Data.xlsx')

print(df.columns)

df.head()

Index(['Text', 'Label'], dtype='object')


,Text,Label
0,i just fell this morning in my kitchen,emergency
1,i see no reason for living,emergency
2,i fell in love when i was a young man,neutral
3,coffee is my reason for living,neutral
4,my afternoon meal is 3 hours late,emergency


In [10]:
df["Text"][:2].tolist()

['i just fell this morning in my kitchen', 'i see no reason for living']

In [22]:
import random

records = df.to_records(index=False)
result = list(records)

random.shuffle(result)

val_list = result[:int(len(result)*.2)]
train_list = result[int(len(result)*.2):]

val_list

[('Without mexican food I will die', 'neutral '),
 ('I laughed so hard that I almost peed my pants. ', 'neutral'),
 ('my son has been hitting me', 'emergency'),
 ('the tv shows are so violent these days', 'neutral '),
 ('i think i hurt my wrist while working on my yard', 'emergency'),
 ('my afternoon meal is 3 hours late', 'emergency'),
 ("It's very common to have some allergies during summer", 'neutral'),
 ('After hearing her story I felt sick to my stomach', 'neutral'),
 ('I slipped in the bathroom', 'emergency')]

In [23]:
train_list

[('i bumped my head with the sidewalk', 'emergency'),
 ('i feel out out breath walking up the stairs', 'neutral'),
 ('on the news i saw that someone was hit by a car', 'neutral'),
 ('i fell down on the driveway', 'emergency'),
 ("I'm having an allergic attack. Help!", 'emergency'),
 ('i see no reason for living', 'emergency'),
 ('i just got run over by a car', 'emergency'),
 ('I will die if someone tickled me that way ', 'neutral '),
 ('the red sox got run over by the yankees', 'neutral'),
 ('I cut myself while cooking and its bleeding', 'emergency'),
 ('i am out of my medications', 'emergency'),
 ('coffee is my reason for living', 'neutral'),
 ('I was supposed to go to the grocery but it slipped my mind. ', 'neutral '),
 ('i fell in love when i was a young man', 'neutral'),
 ('i spent all night throwing up', 'emergency'),
 ('i feel out of breath just walking to the kitchen', 'emergency'),
 ('my dog hasnt been eating', 'neutral '),
 ('I like the show how to get away with murder', 'neut

In [25]:
random.choices(train_list, k=4)

[('I like the song Killing me softly ', 'neutral'),
 ('i bumped into my neighbor', 'neutral'),
 ('My house is being attacked', 'emergency'),
 ('I fell asleep on the couch ', 'neutral ')]

In [58]:
def end_punctuation(utter):
    
    if utter[-1] not in ["?","!","."]:
        utter+="."
        
    return utter

def get_detection_prompt(
    query_text,
    few_shot_list,
    example_symbol = 'Text: ', 
    label_symbol = 'Label: ', 
    sample_separator = '###', 
    join_separator = '\n', 
):
    
    detect_prompt = []
    
    for xy in few_shot_list:
        
        x = end_punctuation(xy[0].strip())
        y = end_punctuation(xy[1].strip())
        
        detect_prompt.append(example_symbol+x)
        detect_prompt.append(label_symbol+y)
        detect_prompt.append(sample_separator)

    query_text = end_punctuation(query_text.strip())
    
    detect_prompt.append(example_symbol+query_text)
    detect_prompt.append(label_symbol)
    
    return join_separator.join(detect_prompt)

In [60]:
concat_prompt = ''

for i in range(len(train_list)):

  few_shot_list = random.choices(train_list, k=2)

  prompt = get_detection_prompt(
      query_text = train_list[i][0],
      few_shot_list = few_shot_list,
  )

  prompt += train_list[i][1]
  concat_prompt += prompt+'.\n###\n'

concat_prompt

"Text: that music makes me feel like throwing up.\nLabel: neutral.\n###\nText: my legs have been swollen for years.\nLabel: neutral.\n###\nText: i bumped my head with the sidewalk.\nLabel: emergency.\n###\nText: that music makes me feel like throwing up.\nLabel: neutral.\n###\nText: i bumped into my neighbor.\nLabel: neutral.\n###\nText: i feel out out breath walking up the stairs.\nLabel: neutral.\n###\nText: I was supposed to go to the grocery but it slipped my mind.\nLabel: neutral.\n###\nText: I like the song Killing me softly.\nLabel: neutral.\n###\nText: on the news i saw that someone was hit by a car.\nLabel: neutral.\n###\nText: i feel out out breath walking up the stairs.\nLabel: neutral.\n###\nText: i bumped my head with the sidewalk.\nLabel: emergency.\n###\nText: i fell down on the driveway.\nLabel: emergency.\n###\nText: i fell down on the driveway.\nLabel: emergency.\n###\nText: i see no reason for living.\nLabel: emergency.\n###\nText: I'm having an allergic attack. Help

In [3]:
# load dependencies into our python environment
%%capture
! pip install transformers[sentencepiece]

In [63]:

#huggingface transformers
import transformers
print('transformers.__version__', transformers.__version__)
from transformers import set_seed
from transformers import AutoTokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel

pretrained_model = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model)


transformers.__version__ 4.20.1


In [92]:
#torch libs
import torch
print('torch.__version__', torch.__version__)
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

#data manupulation libs
import numpy as np

class EmotionDataset(Dataset):

    def __init__(self, tuple_list, tokenizer, chunk_size = 256):

        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        self.input_token_lens = []

        def get_prompt(tuple_list, i):

            few_shot_list = random.choices(tuple_list, k=4)

            prompt = get_detection_prompt(
                query_text = tuple_list[i][0],
                few_shot_list = few_shot_list,
            )

            prompt += tuple_list[i][1]

            encodings_dict = tokenizer(prompt, return_tensors="pt")

            input_ids = encodings_dict['input_ids']    
            attention_mask = encodings_dict['attention_mask']
            input_token_len = input_ids.shape[1]

            return input_ids, attention_mask, input_token_len

        for i in range(len(tuple_list)):
            unfilled_chunk = True

            input_ids = torch.zeros(1, 0)
            attention_mask = torch.zeros(1, 0)
            input_token_len = 0

            while unfilled_chunk:

              input_ids_, attention_mask_, input_token_len_ = \
                get_prompt(tuple_list, i)
              self.input_token_lens.append(input_token_len_)

              input_token_len += input_token_len_
              input_ids = torch.cat([input_ids, input_ids_], dim=1)
              attention_mask = torch.cat([attention_mask, attention_mask_], dim=1)

              if input_token_len > chunk_size:
                input_ids = input_ids[:,:chunk_size]
                attention_mask = attention_mask[:,:chunk_size]
                unfilled_chunk = False

            self.input_ids.append(input_ids)
            self.attn_masks.append(attention_mask)
            self.labels.append(input_ids)

        print('mean', np.mean(self.input_token_lens), 'min', min(self.input_token_lens), 'max', max(self.input_token_lens))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx], self.labels[idx]

torch.__version__ 1.11.0+cu113


In [93]:
traindata = EmotionDataset(train_list, tokenizer, chunk_size = 128)

mean 90.1 min 81 max 100


In [97]:
tokenizer.batch_decode(traindata.__getitem__(1)[0])[0]

"Text: the red sox got run over by the yankees.\nLabel: neutral.\n###\nText: i fell down on the driveway.\nLabel: emergency.\n###\nText: i bumped my head with the sidewalk.\nLabel: emergency.\n###\nText: i like my toast slightly burned.\nLabel: neutral.\n###\nText: i feel out out breath walking up the stairs.\nLabel: neutralText: i will kill for some nachos.\nLabel: neutral.\n###\nText: I'm having an allergic attack. Help!\nLabel: emergency.\n###"